# Paper Review 3: Quality Minus Junk

By: Chengyi (Jeff) Chen

### Table of Contents
1. [Data, Methodology, and Quality Measures](#1)
2. [Ex Ante Quality Forecasts Fundamentals](#2)
3. [The Price of Quality](#3)
4. [The Return of Quality Stocks](#4)
5. [Quality Minus Junk](#5)
6. [The Time Variation of the Price of Quality: Predicting QMJ](#6)
7. [Quality at a Reasonable Price](#7)
8. [QMJ on the Right-Hand-Side of a Factor Model](#8)

---
## 1. Data, Methodology, and Quality Measures<a id="1"></a>

### Data Sources

#### Long Sample

- U.S. only stocks

- Jun 1956 - Dec 2012

#### Broad Sample

- ALL available common stocks om 24 developed markets in the [CRSP](http://www.crsp.org/) and [XpressFeed Global database](https://www.spglobal.com/marketintelligence/en/solutions/xpressfeedtm)

- Jan 1986 - Dec 2012

### Quality Score

\begin{align}
    x &: \text{variable of interest} \\
    r &: \text{vector of ranks}, r_i = rank(x_i) \\
    \mu_r &: \text{cross sectional mean} \\
    \sigma_r &: \text{cross sectional standard deviation } \\
    z(x) &= \frac{(r - \mu_r)}{\sigma_r} \\
\end{align}

<table style="width:100%">
  <tr>
    <th style="text-align:center">Proxy</th>
    <th style="text-align:center">Definition</th>
    <th style="text-align:center">Formula</th>
  </tr>
  <tr>
    <td style="text-align:center">Profitability</td>
    <td style="text-align:center">Average of $z$-scores of profitability measures</td>
    <td style="text-align:center">
        \begin{array}
            \\
            z( \\
            z_{\text{gross profits over assets (GPOA)}} \\
                + z_{\text{return on equity (ROE)}} \\
                + z_{\text{return on assets (ROA)}} \\
                + z_{\text{cash flow over assets (CFOA)}} \\
                + z_{\text{gross margin (GMAR)}} \\
                + z_{\text{fraction of earnings composed of cash (i.e. low accruals, ACC)}} \\
            ) \\
        \end{array}
    </td>
  </tr>
  <tr>
    <td style="text-align:center">Growth</td>
    <td style="text-align:center">5-year prior growth in profitability</td>
    <td style="text-align:center">
        \begin{array}
            \\
            z( \\
            z_{\Delta\text{gross profits over assets (GPOA)}} \\
                + z_{\Delta\text{return on equity (ROE)}} \\
                + z_{\Delta\text{return on assets (ROA)}} \\
                + z_{\Delta\text{cash flow over assets (CFOA)}} \\
                + z_{\Delta\text{gross margin (GMAR)}} \\
                + z_{\Delta\text{fraction of earnings composed of cash (i.e. low accruals, ACC)}} \\
            ) \\
        \end{array}
    </td>
  </tr>
  <tr>
    <td style="text-align:center">Safety</td>
    <td style="text-align:center">Average of $z$-scores of safety measures</td>
    <td style="text-align:center">
        \begin{array}
            \\
            z( \\
            z_{\text{CAPM }\beta\text{ (BAB)}} \\
                + z_{\text{idiosyncratic volatility (IVOL)}} \\
                + z_{\text{leverage (LEV)}} \\
                + z_{\text{bankruptcy risk (O-score)}} \\
                + z_{\text{bankruptcy risk (Z-score)}} \\
                + z_{\text{ROE volatility (EVOL)}} \\
            ) \\
        \end{array}
    </td>
  </tr>
  <tr>
    <td style="text-align:center">Payout</td>
    <td style="text-align:center">Average of $z$-scores of payout measures</td>
    <td style="text-align:center">
        \begin{array}
            \\
            z( \\
            z_{\text{equity net issuance (EISS)}} \\
                + z_{\text{debt net issuance (DISS}} \\
                + z_{\text{total net payout over profits (NPOP)}} \\
            ) \\
        \end{array}
    </td>
  </tr>
  <tr>
    <td style="text-align:center">Quality</td>
    <td style="text-align:center">Average of $z$-scores of 4 proxies</td>
    <td style="text-align:center">
        \begin{array}
            \\
            z( \\
            Profitability \\
            + Growth \\
            + Safety \\
            + Payout \\
            ) \\
        \end{array}
    </td>
  </tr>
</table>

### Portfolios

NYSE Breakpoints: 

- https://quant.stackexchange.com/questions/49651/what-is-nyse-breakpoint-as-used-by-fama-french

- http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/six_portfolios.html

- https://quant.stackexchange.com/questions/41729/fama-french-sorting


#### Quality-sorted Portfolios

U.S. Long Sample

```python
    def country_quality_sorted_portfolio(country):
        """Returns 1 value-weighted quality-sorted portfolio for the country
        """
        # 1. Sort each stock in the investable universe by quality score defined above
        # 2. Partition sorted stocks into 10 portfolios based on NYSE breakpoints
        # 3. Weigh each portfolio by value and combine into a single country portfolio
        # 4. Rebalance at the end of each calendar month
```

Global Broad Sample

```python
    def global_quality_sorted_portfolio(country):
        """Returns 1 country-total-(lagged)-market-cap-weighted portfolio
        """
        # 1. Get the quality-sorted portfolios for each country
        portfolios = []
        for country in broad_sample.countries:
            portfolios.append(country_quality_sorted_portfolio(country))
        # 2. Weigh each country portfolio by the country’s total (lagged) market capitalization and combine into 1 global portfolio
```

#### Quality-Minus-Junk factor portfolio (QMJ)

U.S. Long Sample

```python
    def country_qmj_portfolio(country):
        """Returns 1 value-weighted quality-minus-junk factor portfolio for the country
        
        Resources: 
        - Kenneth French's website: http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/six_portfolios.html
        """
        # 1. Partition stocks in each country into 2, Big and Small, using median NYSE Market equity as breakpoint for U.S. and 80th percentile of market equity for International stocks
        # 2. Further partition each size partition into 3 groups sorted on quality (similar to the Fama-french sorting)
        # 3. QMJ factor return is the avg return on the 2 high quality portfolios - avg return on the 2 low quality portfolios
        QMJ_return = 0.5*(small_quality + big_quality) - 0.5*(small_junk + big_junk)
        return QMJ_return
```

Global Broad Sample

```python
    def global_qmj_portfolio(country):
        """Returns 1 country-total-(lagged)-market-cap-weighted portfolio
        """
        # 1. Get the quality-minus-junk factor portfolios for each country
        portfolios = []
        for country in broad_sample.countries:
            portfolios.append(country_qmj_portfolio(country))
        # 2. Weigh each country portfolio by the country’s total (lagged) market capitalization and combine into 1 global portfolio
        
```

---
## 2. Ex Ante Quality Forecasts Fundamentals<a id="2"></a>

- Quality firms today remain high quality firms + 120Months

---
## 3. The Price of Quality<a id="3"></a>

#### Fama-Macbeth Regressions

\begin{align}
    P^i_t &: z_{\text{stock i's market-to-book (MB) ratio at time t}}
\end{align}

1. Regress the Time Series of $P^i_t$ against Time Series of $Quality^i_t$ (defined in section 1) to get overall quality score exposure of each stock $i$'s market-to-book (MB) ratio.
\begin{align}
    \hat{P^i_t} &= a + bQuality^i_t + \epsilon^i_t \\
    b &: \text{if quality improves by one standard deviation, then the price-to-book increases by }b\text{ standard deviations} \\  
\end{align}
2. Regress $P^i_t$ of all the stocks in the Portfolio @ time $t$ against the overall quality score exposure of each stock $i$'s market-to-book (MB) ratio

The fama-macbeth regressions are also run by regressing the time series of $P^i_t$ on different combinations of:

- $Size$: z-score of the stock’s market equity (ME)

- $Ret(t-12,t)$: stock's return in the prior year

- Industry fixed effects

- Country fixed effects

Price of Quality $b$'s Interpretation with the inclusion of industry / country fixed effects: 

- if quality improves by one standard deviation above its country-industry mean, then the price-to-book increases by b standard deviations above its country-industry mean.)

Summary:

- Price of quality is statistically significantly positive from both US and global samples across different combinations of country / industry fixed effects, and with and without the other covariates $Size$, and $Ret(t-12, t)$

- Quality does not explain a large portion of the cross sectional variance in prices (low average $R^2$), meaning that there are other big forces driving the prices besides quality alone

- When regressing against separate components of the quality measurement, we find:

    - Price of $Profitability$ is $> 0$
    
    - Price of $Growth$ is $> 0$
    
    - Price of $Safety$ is $< 0$ in univariate regression, but $> 0$ when regressed with the other quality measure proxies
    
        - Possibly explained by: If investors are constrained from leveraging, risky assets command higher prices (and lower returns) while safe assets have lower prices (and higher returns).
    
    - Price of $Payout$ is $< 0$

---
## 4. The Return of Quality Stocks<a id="4"></a>

Possible theories as to why Quality is limited in explaining Prices:

1. Limited Market Efficiency

    - Implication: High-quality stocks have high risk-adjusted returns than low-quality stocks as investors are underpricing high quality characteristics
    
    - Most agreeable with the analysis

2. Market uses superior quality measures

    - Implication: No relation between our measured quality and ex post returns or at least a greatly attenuated one
    
    - Results clearly show that there is  a statistically significant relationship ($\beta$ coefficient in regression) between quality and ex post returns as  excess returns increase almost monotonically in quality such that high-quality stocks outperform low-quality stocks (difference between the highest and lowest deciles and the associated t-statistic), disproving this theory. 

3. Quality is linked to risk in a way not captured by our safety measure

    - Implication: Univariate relation between quality and future returns which is reduced or eliminated by an effective risk model
    
    - When we control for market risk and other factor exposures, the outperformance in the alpha of high-quality stocks is in fact even larger meaning that High-quality stocks have lower market exposures than junk stocks, disproving this theory.

---
## 5. Quality Minus Junk<a id="5"></a>

#### Correlation Analysis of monthly returns of $QMJ, Profitability, Safety, Growth, Payout$ factors

- Pairwise correlations mostly $> 0$

    - High magnitude in one proxy for quality is usually associated with a different proxy even though it didn't necessarily have to be

- Correlation between $Growth$ factor and $Payout$ factor $< 0$

    - Higher payout is naturally associated with lower growth

#### Factor Loadings of $MKT, SMB, HML, UMD$ factors with $QMJ, Profitability, Safety, Growth, Payout$ factors

- QMJ factor has a significantly negative market and size exposures

    - QMJ is long low-beta and large stocks, while being short high-beta small ones
    
- Profitability, safety, and growth factors have significantly negative HML loadings in the U.S. and global samples

    - we expect that high-quality stocks have high prices while the value factor HML is long cheap stocks, we would expect a negative HML loading
    
- The payout factor has a positive loading in the U.S. and global samples

    - Positive payout loading could be driven by cheap stocks endogenously choosing a low payout. 

#### Performance of the QMJ factor across countries

- QMJ factor delivers positive returns and alphas in all but one of the 24 countries

    - Negative in New Zealand, one of the smallest countries in market capitalization and number of stocks

#### Performance of the QMJ factor over time

- QMJ factor has consistently delivered positive excess returns and risk-adjusted returns over time 

#### Driver of large and significant 1- , 3- and 4-factor abnormal returns in QMJ portfolios that are long high quality stocks and short junk stocks

- Evidence points away from compensation for tail risk 

     - The strong return in extreme down markets is consistent with a flight to quality (or at least profitability), meaning that taking on tail risk during economic downturns is not plausible evidence that QMJ returns are driven by compensation for tail risk
     
     - Hence, market mispricing might be more plausible a theory.

---
## 6. The Time Variation of the Price of Quality: Predicting QMJ<a id="6"></a>

- Variation of the price of quality is not pure noise but, rather, reflects changes in the market pricing of quality characteristics, generating variation in QMJ returns.

---
## 7. Quality at a Reasonable Price<a id="7"></a>

- Fair price of quality is above the level paid by the market, meaning that quality is currently undervalued.

---
## 8. QMJ on the Right-Hand-Side of a Factor Model<a id="8"></a>

### Regressing each non-market factor in 4-factor model ($SMB, HML, UMD$) against $QMJ$ factor and the other factors

#### $SMB$

- $SMB$ has a very large negative exposure to $QMJ$.

#### $HML$

- $HML$ has a negative loading on $QMJ$

    - Cheap stocks (with high book-to-market) are naturally lower quality than expensive stocks
    
    - Implication: controlling for $QMJ$ increases the alpha of $HML$, strengthening the value effect. 
    
    
#### $UMD$

- $UMD$ has positive loading on QMJ, which is significant in the global but not U.S. sample. Controlling for this exposure to quality lowers the alpha of UMD, but the momentum effect remains highly significant in both samples